# Text summarization of Hotel Reviews using Huggingface Transoformers T5 and Bart

In [4]:
import pandas as pd

I have kept my review files (1141) files in google drive for pre-processing on given location.

In [5]:
fileNames = range(1,1142)
filePath = '/content/drive/My Drive/Colab Notebooks/hotel_reviews/'

In [6]:
import re

*   Review files contains data in comma seperated format, where date, review_title and review_text present respectively in each file
*   Only keeping review_text with length of > 100
*   Total reviews as shown below are 76634



In [ ]:
reviews = []
filteredReviews=[]
for fl in fileNames:
  with open(str(filePath)+str(fl),'r', encoding='windows-1252') as f:
    splits = re.split(r'\n+',f.read())  
    for s in splits:
      reviews.append(re.split("\t+",s))

for r in reviews:
  reviewText = ''
  for rev in range(len(r)):
    if rev == 0 or rev == 1:
      pass
    else:
      reviewText+=r[rev]
  if len(reviewText)>100:
    filteredReviews.append(reviewText)

print(filteredReviews[0])
print(filteredReviews[1])
print(len(filteredReviews))

We spent two separate nights at this hotel as part of our tour of India.As others have said, it is heavily used by and geared to tour groups, There are a lot of arrivals and departures in the night and early morning, which may be disturbing to individual travellers.This is a good hotel though. Our room was pleasant and the bathroom clean. The food was good and the service very friendly and welcoming, except at the front desk where staff were a little abrupt.The gardens are very well maintained, quiet and peaceful. On our second visit our enjoyment of them was increased by the beautiful decorations and fairy lights for a wedding reception to be held that evening.We would not hesitate to return for an overnight stay.
The Ashok is reasonably close to the airport. Getting there is rather odd, you have to drive down a dirt track and the hotel is at the end - we were in transit, so it didn't matter much, but you wouldn't want to stay there if you were on a trip to see the sights of Delhi. Th

Keeping pre-processed reviews in separate file (reviews.csv)

In [ ]:
reviewsCollection = pd.DataFrame(filteredReviews,columns=['review'])
reviewsCollection.to_csv(filePath+'reviews.csv')

In [ ]:
reviewsDataPath = filePath+'reviews.csv'
df = pd.read_csv(reviewsDataPath,index_col=0)
df.head()

,review
0,We spent two separate nights at this hotel as ...
1,The Ashok is reasonably close to the airport. ...
2,useful stopover if visiting Delhi. good pool a...
3,On a five star scale rating this hotel would q...
4,This is one of the nicer hotels in Delhi and i...


Installing Huggingface Tranformers by following command, though almost everything is available in Google Colab, just to make sure it works fine so ran it.

In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 3.1MB/s 
     |████████████████████████████████| 890kB 8.2MB/s 
     |████████████████████████████████| 1.1MB 10.8MB/s 
     |████████████████████████████████| 3.0MB 20.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=42384d8eaa6fab4442473ca2cc3335855826594c2b8507f8813e6a9b1483f6e5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


We are going to compare T5 and Bart summarization results with each other, as we dont have human created summarizations for this reviews.

# Initialize and summarizing with T5 Summarizer
I am passing all the sentences to encoder one after another and in-between getting generated text and decoding them which is our output of summarization for each sentenece.

Processing only 2001 records as processing all the records does not make sense as I am only utilizing the transformer rather than training it on a task, and it is also time-consuming.


---


    Warning shown below is questionable, as it only allows limit text to be processed. Checked on forums during development of this project, this warning might get removed in future but it is not impacting our output as of now.

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

summarized_text = []

for review in df['review'][:2001].values:
  preprocess_text = review.strip().replace("\n","")
  t5_inputText = "summarize: "+preprocess_text

  tokenized_text = tokenizer.encode(t5_inputText, return_tensors="pt").to(device)

  summary_ids = model.generate(tokenized_text,num_beams=4, no_repeat_ngram_size=2,
                              min_length=30, max_length=300,early_stopping=True).to(device)

  summarized_output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  summarized_text.append(summarized_output)

Some weights of the model checkpoint at t5-small were not used when initializing T5ForConditionalGeneration: ['encoder.block.0.layer.0.layer_norm.bias', 'encoder.block.0.layer.1.layer_norm.bias', 'encoder.block.1.layer.0.layer_norm.bias', 'encoder.block.1.layer.1.layer_norm.bias', 'encoder.block.2.layer.0.layer_norm.bias', 'encoder.block.2.layer.1.layer_norm.bias', 'encoder.block.3.layer.0.layer_norm.bias', 'encoder.block.3.layer.1.layer_norm.bias', 'encoder.block.4.layer.0.layer_norm.bias', 'encoder.block.4.layer.1.layer_norm.bias', 'encoder.block.5.layer.0.layer_norm.bias', 'encoder.block.5.layer.1.layer_norm.bias', 'encoder.final_layer_norm.bias', 'decoder.block.0.layer.0.layer_norm.bias', 'decoder.block.0.layer.1.layer_norm.bias', 'decoder.block.0.layer.2.layer_norm.bias', 'decoder.block.1.layer.0.layer_norm.bias', 'decoder.block.1.layer.1.layer_norm.bias', 'decoder.block.1.layer.2.layer_norm.bias', 'decoder.block.2.layer.0.layer_norm.bias', 'decoder.block.2.layer.1.layer_norm.bias

Token indices sequence length is longer than the specified maximum sequence length for this model (761 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (680 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (746 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (804 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (600 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [ ]:
len(summarized_text)

2001

Saving T5 summarizer output in a file for further use and prevent re-processing of entire previous sequence.

In [ ]:
t5_output = pd.DataFrame(summarized_text,columns=['summarization'])
t5_output.to_csv(filePath+'t5_output.csv')
t5_output.head()

,summarization
0,we spent two separate nights at this hotel as ...
1,the hotel and rooms (although large) are chara...
2,road works when i was there so getting to and ...
3,this hotel would qualify as a high end 3 star ...
4,this is one of the nicer hotels in Delhi and i...


# Initialize and summarizing with Bart Summarizer
I am passing all the sentences to generate method decoding them for our output of summarization for each sentenece.

I have kept 300 as a maximum length of sentence to be generated on purpose as in T5 Summarizer.

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

summarized_text = []

for review in df['review'][:2001].values:
  preprocess_text = review.strip().replace("\n","")
  bart_inputText = "summarize: "+preprocess_text

  tokenized_text = tokenizer([review], truncation=True, return_tensors='pt')

  summary_ids = model.generate(tokenized_text['input_ids'], num_beams=4,
                               min_length=30, max_length=300, early_stopping=True)
  for si in summary_ids:
    summarized_text.append(tokenizer.decode(si, skip_special_tokens=True,
                                            clean_up_tokenization_spaces=False))  

Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-large-cnn and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
summarized_text[:3]
len(summarized_text)

2001

In [ ]:
bart_output = pd.DataFrame(summarized_text,columns=['summarization'])
bart_output.to_csv(filePath+'bart_output.csv')
bart_output.head()

,summarization
0,The hotel is heavily used by and geared to tou...
1,The Ashok is reasonably close to the airport. ...
2,useful stopover if visiting Delhi. good pool a...
3,On a five star scale rating this hotel would q...
4,The hotel is located near both the domestic an...


# I am using Rouge evaluation for our results.
Using python implementation of Rouge-l metric found on this github repository:

https://github.com/pcyin/PyRouge

In order to use this in Google Colab I had to run below commands for custom library import.

     Process might be different, if you are running on desktop.

In [40]:
%cd './drive/My Drive/Colab Notebooks/PyRouge'
%pwd

[Errno 2] No such file or directory: './drive/My Drive/Colab Notebooks/PyRouge'
/content/drive/My Drive/Colab Notebooks/PyRouge


'/content/drive/My Drive/Colab Notebooks/PyRouge'

In [43]:
import imp 
pyrouge = imp.new_module('pyrouge')

In [27]:
t5_output = pd.read_csv(filePath+'t5_output.csv',index_col=0)
bart_output = pd.read_csv(filePath+'bart_output.csv',index_col=0)
[len(bart_output),len(t5_output)]

[2001, 2001]



*   Rouge-l implemented in the library returns precision, recall and f1 score for each sentence.
*   I am keeping all of them in lists for both T5 and Bart for further processing.



In [69]:
t5_to_bart_prec = []
t5_to_bart_rec = []
t5_to_bart_f1 = []
bart_to_t5_prec = []
bart_to_t5_rec = []
bart_to_t5_f1 = []

from pyrouge import Rouge
ro = Rouge()

for location in range(2001):
  [p,r,f] = ro.rouge_l([t5_output['summarization'][location]],[bart_output['summarization'][location]])
  t5_to_bart_prec.append(p)
  t5_to_bart_rec.append(r)
  t5_to_bart_f1.append(f)

  [p,r,f] = ro.rouge_l([bart_output['summarization'][location]], [t5_output['summarization'][location]])
  bart_to_t5_prec.append(p)
  bart_to_t5_rec.append(r)
  bart_to_t5_f1.append(f)

*  As purpose of this project was just to compare two summarizer, so evaluation is different.

*  In a real world, we need to fine-tune this transformer on specific task like summarization and then need human created summarizations to compare with system generated summarizations.

*  So, here I am taking mean of all precision, recall and f1 score of T5 and Bart summarization as shown below.

*  As expected both of them are giving same 62% f1 score.


    scores would be same because we are comparing results of T5 to Bart and vice-versa.

In [71]:
t5_precision = sum(t5_to_bart_prec)/len(t5_to_bart_prec)
t5_recall = sum(t5_to_bart_rec)/len(t5_to_bart_rec)
t5_f1_score = sum(t5_to_bart_f1)/len(t5_to_bart_f1)

bart_precision = sum(bart_to_t5_prec)/len(bart_to_t5_prec)
bart_recall = sum(bart_to_t5_rec)/len(bart_to_t5_rec)
bart_f1_score = sum(bart_to_t5_f1)/len(bart_to_t5_f1)

print('SUMMARIZATION RESULTS OF 2000 SENTENCES FROM HOTEL REVIEWS')
print('*'*40+' T5 SUMMARIZATION SCORE '+'*'*40)
print('PRECISION | RECALL | F1 SCORE')
print(str(round(t5_precision,2))+'\t   ', str(round(t5_recall, 2))+'\t', str(round(t5_f1_score, 2)))
print('-'*80)

print('*'*40+' BART SUMMARIZATION SCORE '+'*'*40)
print('PRECISION | RECALL | F1 SCORE')
print(str(round(bart_precision, 2))+'\t   ', str(round(bart_recall, 2))+'\t', str(round(bart_f1_score, 2)))
print('-'*80)

SUMMARIZATION RESULTS OF 2000 SENTENCES FROM HOTEL REVIEWS
**************************************** T5 SUMMARIZATION SCORE ****************************************
PRECISION | RECALL | F1 SCORE
0.61	    0.66	 0.62
--------------------------------------------------------------------------------
**************************************** BART SUMMARIZATION SCORE ****************************************
PRECISION | RECALL | F1 SCORE
0.66	    0.61	 0.62
--------------------------------------------------------------------------------
